## Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection

Use the code you just worked out to build a protocol and an "echo" bot. You don't actually have to do the network part of this if you don't want; just simulate that. The protocol is:

```
A->B
    Send "p", "g", "A"
B->A
    Send "B"
A->B
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
B->A
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv
```

(In other words, derive an AES key from DH with SHA1, use it in both directions, and do CBC with random IVs appended or prepended to the message). 

In [1]:
g = 2
p = int(
"ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024"
"e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd"
"3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec"
"6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f"
"24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361"
"c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552"
"bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff"
"fffffffffffff", 16)
p

2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919

In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from secrets import token_bytes

In [2]:
def encrypt(key:bytes, message:bytes):
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(message, cipher.block_size))
    return ciphertext, cipher.iv

In [3]:
key = token_bytes(16)

In [4]:
ciphertext, iv = encrypt(key, b'foobar')
print(ciphertext.hex(), iv.hex())

7337aa8ed8008dc77fe97e212850d1d9 78a38da95438610ac69679df6272fa8e


In [5]:
def decrypt(key:bytes, ciphertext:bytes, iv:bytes):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ciphertext), AES.block_size)

In [6]:
decrypt(key, ciphertext, iv)

b'foobar'

Now implement the following MITM attack:

```
A->M
    Send "p", "g", "A"
M->B
    Send "p", "g", "p"
B->M
    Send "B"
M->A
    Send "p"
A->M
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
M->B
    Relay that to B
B->M
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv
M->A
    Relay that to A
```

M should be able to decrypt the messages. "A" and "B" in the protocol --- the public keys, over the wire --- have been swapped out with "p". Do the DH math on this quickly to see what that does to the predictability of the key.

Decrypt the messages from M's vantage point as they go by. 

In [2]:
from random import randrange

Alice sends DH parameters $p$ and $g$ and public key $A$ to Bob.

In [3]:
a = randrange(1, p)
A = pow(g, a, p)

MITM attacker intercepts and replaces value of $A$ with a chosen "bogus" value and forwards $p$, $g$, and $A = p$ to Bob.

In [4]:
A = p

Bob receives $A = p$ and sends public key $B$ to Alice.

In [5]:
b = randrange(1, p)
B = pow(g, b, p)

MITM attacker intercepts and replaces with $B = p$ and forwards to Alice.

In [6]:
B = p

Alice calculates shared secret $s = B^a\mod{p}$.

In [7]:
s = pow(B, a, p)

Similarly, Bob calculates $s = A^b\mod{p}$.

In [8]:
assert s == pow(A, b, p)

Note, MITM attacker has forced the shared secret $s = 0$ because whatever the values of $a$ or $b$, $p^a$ and $p^b$ will be a multiple of $p$ which will be congruent to $0$ modulo $p$.

In [9]:
s

0

Note that you don't actually have to inject bogus parameters to make this attack work; you could just generate Ma, MA, Mb, and MB as valid DH parameters to do a generic MITM attack. But do the parameter injection attack; it's going to come up again. 